In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
import time

In [2]:
def init_browser():
    # @NOTE: Replace the path with your actual path to the chromedriver
    executable_path = {"executable_path": "chromedriver.exe"}
    return browser("chrome", **executable_path, headless=False)

In [3]:
# def scrape_info():
#browser = Browser('chrome')
#browser = init_browser()
news = {}

In [4]:
# Visit mars.nasa.gov/newa
browser = Browser('chrome')
#news_url = "https://mars.nasa.gov/news/"
#browser.visit(news_url)
browser.visit("https://mars.nasa.gov/news/")
time.sleep(5)

In [5]:
# Scrape page into Soup
html = browser.html
soup = bs(html, "html.parser")

In [6]:
# Get the news title
news_title = soup.find('div', class_='content_title').a.text
print(news_title)

NASA's Treasure Map for Water Ice on Mars


In [7]:
#Get the paragraph of news
paragraph = soup.find('div', class_='article_teaser_body').text
print(paragraph)

# news_p=" "

A new study identifies frozen water just below the Martian surface, where astronauts could easily dig it up.


In [8]:
# Store data in a dictionary
news= {
        "Title": news_title,
        "Paragraph": paragraph,
        
    }

In [9]:
#visit nasa site for image

browser = Browser('chrome')
url="https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"

browser.visit(url)
time.sleep(5)

In [10]:
#get the full image

full_image_elem = browser.find_by_id('full_image')
full_image_elem.click()
time.sleep(5)

In [11]:
#scrape image into soup

browser.is_element_present_by_text('more info', wait_time=1)
more_info_elem = browser.find_link_by_partial_text('more info')
more_info_elem.click()
time.sleep(5)

In [12]:
#get url for image


soup = bs(browser.html, 'html.parser')
result=soup.find('figure',class_="lede")
featured_image_url="https://www.jpl.nasa.gov"+result.a.img["src"]
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA17357_hires.jpg'

In [13]:
#store image in news

news['featured_image']=featured_image_url

In [14]:
# Visit the Mars Weather twitter account 

browser = Browser('chrome')
url="https://twitter.com/marswxreport?lang=en"

browser.visit(url)
time.sleep(5)

In [15]:
# Find tweet title on mars weather 
soup = bs(browser.html, 'html.parser')
mars_weather = soup.find('div', class_="js-tweet-text-container")
# print(mars_weather)
weather=mars_weather.p.text
print(weather)



InSight sol 374 (2019-12-15) low -97.3ºC (-143.1ºF) high -21.2ºC (-6.1ºF)
winds from the SSE at 5.6 m/s (12.5 mph) gusting to 22.0 m/s (49.1 mph)
pressure at 6.50 hPapic.twitter.com/CPAP5esipU


In [17]:
#Save the tweet text for the weather report as a variable called `mars_weather`.
# Store data in a dictionary
news['mars_weather']=weather

In [18]:
import pandas as pd
df = pd.read_html('https://space-facts.com/mars/')[0]
df.columns=['description', 'value']
df.set_index('description', inplace=True)
df

,value
description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [19]:
data=df.to_html()
data

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>value</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\

In [20]:
news['facts']=data

In [22]:
url='https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)


hemisphere_image_urls = []
# First, get a list of all of the hemispheres
links = browser.find_by_css("a.product-item h3")
# Next, loop through those links, click the link, find the sample anchor, return the href
for i in range(len(links)):
    hemisphere = {}
    # We have to find the elements on each loop to avoid a stale element exception
    browser.find_by_css("a.product-item h3")[i].click()
    time.sleep(1)
    # Next, we find the Sample image anchor tag and extract the href
    sample_elem = browser.find_link_by_text('Sample').first
    hemisphere['img_url'] = sample_elem['href']
    # Get Hemisphere title
    hemisphere['title'] = browser.find_by_css("h2.title").text
    # Append hemisphere object to list
    hemisphere_image_urls.append(hemisphere)
    # Finally, we navigate backwards
    browser.back()
    
print(hemisphere_image_urls)



[{'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg', 'title': 'Cerberus Hemisphere Enhanced'}, {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg', 'title': 'Schiaparelli Hemisphere Enhanced'}, {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg', 'title': 'Syrtis Major Hemisphere Enhanced'}, {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg', 'title': 'Valles Marineris Hemisphere Enhanced'}]


In [23]:
news['hemisphere']=hemisphere_image_urls